<a href="https://colab.research.google.com/github/seyeon0815/Project/blob/main/%ED%98%BC%EA%B3%B5%EB%A8%B8%EC%8B%A0_chapter_5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치
- 키워드 : 하이퍼 파라미터
- 데이터가 작을 때 주로 사용

# 검증 세트
- 테스트 세트 (일회성)
- 훈련 데이터를 훈련 데이터 + 검증 데이터로 재분할


## 현실
- 테스트 데이터가 별도로 존재하지 않음
- 전체 데이터 = 훈련(6) : 검증(2) : 테스트(2)
 + 테스트 데이터는 모르는 데이터로 생각
 + 테스트 데이터는 마지막에 일회성으로 적용

In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
# 훈련 세트와 테스트 세트를 나누는 것은 이전과 동일
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [ ]:
# 훈련세트 sub_input/target 과 검증세트 val_input/target 으로 다시 나누기
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size = 0.2, rand om_state=42
) 
# 여기서도 test_size는 20%로 val_input 만들기

In [ ]:
print(sub_input.shape, val_input.shape, test_input.shape) #훈련데이터세트, 검증 데이터세트, 테스트 데이터세트

(4157, 3) (1040, 3) (1300, 3)


# 모델 만든 후 평가
- 결정트리 분류 모델 사용

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
#과대적합 발생(훈련세트 점수 > 검증세트 점수) => 매개변수를 바꿔서 더 좋은 모델을 찾아야 함

0.9971133028626413
0.864423076923077


# 교차 검증
- 교차 검증의 목적 : 좋은 모델을 만들기 위해
 + 좋은 모델 != 성능 좋은 모델
 + 좋은 모델 = 과대적합이 아닌 모델 = 모형의 오차가 적은 모델 = 안정적인 모델(어떤 데이터가 들어와도 ㅡ%를 유지하는 모델)

 - 교차 검증이란?
 1. 검증 세트/ 훈련 세트/ 훈련 세트
 2. 훈련 세트/ 검정 세트/ 훈련 세트
 3. 훈련 세트/ 훈련 세트/ 검증 세트
+ 훈련세트끼리 모델 훈련, 검증 세트로 모델 평가 -> 각 단계에서 검증 점수를 평균
+ 각각의 세트를 '폴드'라고 함
+ 위는 훈련 세트를 세 부분으로 나눠서 교차 검증을 수행한 3-폴드 교차 검증 = k-폴드 교차검증(k-fold cross validation)
+ 보통 5-폴드/10-폴드 교차 검증을 많이 사용 ->데이터의 80~90%까지 훈련에서 사용 가능
+ kfold의 경우 일정한 간격으로 잘라서 사용 -> 0,1,2 를 0,1/2 또는 0,2/1로 학습했을 때 2, 1 의 값을 도출 할 수 없음 -> 이를 보완하기 위해 StratifiedKfold 등장

- StratifiedKFold
+ target에 속성값의 개수를 동일하게 가져감으로써 KFold와 같이 데이터가 한쪽으로 몰리는 것을 방지해줌

# 교차 검증 함수
 - cross_validate()함수는 기본적으로 5-폴드 교차 검증 수행

In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)
# fit_time: 모델을 훈련하는 시간
# score_time: 모델을 검증하는 시간
# test_score : 교차 검증 점수 -> 5개 점수를 평균하여 교차 검증의 최종 점수 얻을 수 있음

# 최종 점수 평균 구하기
import numpy as np
print(np.mean(scores['test_score'])) # 교차 검증 점수 5개 평균하여 최종 점수 얻기

{'fit_time': array([0.00845838, 0.00688076, 0.00716376, 0.00725484, 0.00754499]), 'score_time': array([0.00065303, 0.00058985, 0.0005753 , 0.00059128, 0.00064611]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487


- 훈련 세트 섞은 후 10 폴드 교차 검증

In [ ]:
from sklearn.model_selection import StratifiedKFold # 분류 모델일 경우 타깃 클래스를 골고로 나누기 위해 StartifiedKFold 사용 (회귀 모델일 경우 KFOLD 분할기 사용)
# stratify = 계층화화다 / 계층화된 KFold
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)# splitter = 분할기, 교차 검정 시 훈련 세트를 섞어줌/# n_splits = 몇(k)폴드 교차 검증을 할 지/# shuffle = 데이터 섞기
scores = cross_validate(dt, train_input, train_target, cv = splitter) #cv = cross_valicate 교차 검증 #splitter= 폴드를 어떻게 분할할지 -> StratifiedKFold 방법으로

print(np.mean(scores['test_score']))

0.8631895657329183


- 하이퍼 파라미터 튜닝 꼭 하고 싶을 때 랜덤 서치 사용
- 자동으로 잡아주는 라이브러리 사용

- 
- min_samples_split : 자식 노드를 분할하는데 필요한 최소 샘플 수 (default값은 2)

# 그리드 서치(GridSearchCV)
- 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행
 -> cross_validate()함수 호출할 필요 x

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
    'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005] # 탐색할 매개 변수와 탐색할 값의 리스트를 딕셔너리로 만들기
} #min_impurity_decrease = 최소 불순도
dt = DecisionTreeClassifier(random_state =42)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs = -1)
gs.fit(train_input, train_target)
#parameter가 추가 될 수록 시간이 매우 길어짐 -> n_jobs = -1 : 비유)내가 가진 모든 말을 다 써서 마차를 끌겠다는 뜻 , 시간 단축

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
    'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
    'max_depth' : [3, 4, 5, 6, 7]
}
dt = DecisionTreeClassifier(random_state =42)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs = -1)
gs.fit(train_input, train_target)
#parameter가 추가 될 수록 시간이 매우 길어짐 -> n_jobs = -1 : 내가 쓰고 있는 모든 말을 다 써서 마차를 끌겠다는 뜻

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [3, 4, 5, 6, 7],
                         'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
dt = gs.best_estimator_
print(dt)
print(dt.score(train_input, train_target))
print(gs.best_params_) #최적의 parameter 찾아줌

DecisionTreeClassifier(max_depth=7, min_impurity_decrease=0.0005,
                       random_state=42)
0.8830094285164518
{'max_depth': 7, 'min_impurity_decrease': 0.0005}


In [ ]:
print(gs.cv_results_['mean_test_score'])
#0.0001과 3,4,5,6,7/0.0002와 3,4,5,6,7/,,,로 총 25개

[0.84125583 0.84125583 0.84125583 0.84125583 0.84125583 0.85337806
 0.85337806 0.85337806 0.85337806 0.85318557 0.85780355 0.85799604
 0.85857352 0.85857352 0.85838102 0.85645721 0.85799678 0.85876675
 0.85972866 0.86088306 0.85607093 0.85761031 0.85799511 0.85991893
 0.86280466]


# 랜덤서치
- p.252 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있도록 확률 분포 객체를 전달하

In [ ]:
from scipy.stats import uniform, randint
rgen = randint(0,10)
rgen.rvs(10)

array([3, 9, 4, 9, 5, 0, 2, 9, 5, 7])

In [ ]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 92, 102,  94,  99, 108, 113,  98,  98,  97,  99]))

In [ ]:
# p.254
from sklearn.model_selection import RandomizedSearchCV
params = {
    'min_impurity_decrease' : uniform(0.0001, 0.0001),
    'max_depth' : randint(20, 50)
}

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),
                        params, n_iter=100, n_jobs=-1, random_state= 42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f211f4caad0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f211f4ca9d0>},
                   random_state=42)

# 트리의 앙상블
- LightGBM 기억
 + GBM --> XGBoost --> LightGBM
 + 참고 1. 모델 개발 속도가 빨라졌나?
 + 참고 2. 모델의 성능이 좋아졌나?
= TabNet (2019)
 + 딥러닝 컨셉

In [ ]:
# 매개변수란?
def add(a, b): #여기 a = 1, b= 2 라고 쓸 수 있음 이것이 매개변수
  return a+b

  print(add(1,2)) 

  # (db, train_input, train_target, n_iter=-1 ,,,,) #괄호 안에 순서는 x,y,group,,,순이기 때문에 n_iter=-1이라는 매개변수를 작성해줌

  #과대 적합이 일어날 때 매개변수를 바꾸어야 한다 --> max_depth 의 크기를 변경하는 것